# FIT 5148 Assignment 2 Task C Notebook 2

Submitted By:

* Michelle May Teng Ling (29524512)
* Vipul Krishnan M.D. (28104641)

Date: 21/05/2018

## Importing Libraries

In [5]:
# importing libraries
from pyspark import SparkContext
from pyspark.streaming import StreamingContext
from pymongo import MongoClient
import pprint as p

## Receiving and Writing to DB

The server connects to the streaming server and receives the data. It descretize the stream with window size - 1 second so that it can receive all the data sent in a second. 

The Dstream is then converted to RDDs and are sent to the function 'sendDb. The function collects the RDDs and process them to make it in the suitable form dictionary form and insert to the database.

If it receives a finish signal,it stops the operation and closes the contexts.

In [15]:
# creating spark context and streaming context and stream with the specified port
sc = SparkContext("local[2]","deneme")
ssc = StreamingContext(sc, 1)
socket_stream = ssc.socketTextStream("localhost",9999)

# capture incoming stream with window size = 1s
incoming = socket_stream.window( 1 )

# the function which send data to db
def sendDb(x):
    # each line comes as different RDDs,
    # using collect() fucntion to make a list
    x = x.collect()
    
    if (x[0] == 'finished'):
        # stopping the contexts
        global ssc
        global sc
        ssc.stop()
        sc.stop()
        print("Operation completed successfully")
        return 1
    
    # creating a mongo client and db
    client = MongoClient()
    db = client.fit5148_db2
    db = db.streamed
    
    # making dictionary for inserting
    d1 = {}
    
    # initialize firedata as empty list
    d1['Fire Data'] = []
    
    # for each line (or row)
    for line in x:
        # splits the individual attributes
        word = line.split(',')
        
        # if climate data, add directly to main dictionary d1
        if word[0] == 'cdata':
            d1["Station"] = word[1]
            d1['Air Temperature(Celcius)'] = word[2]
            d1['Relative Humidity'] = word[3]
            d1['WindSpeed  (knots)'] = word[4]
            d1['Max Wind Speed'] = word[5]
            d1['MAX'] = word[6]
            d1['MIN'] = word[7]
            d1['Precipitation']  = word[8]
        
        # if fire data, make new dictionaries and append to Fire Data list of main dictionary
        elif word[0] == 'fdata':
            d2 = {}
            d2['Latitude'] = word[1]
            d2['Longitude'] = word[2]
            d2['Surface Temperature (kelvin)'] = word[3]
            d2['Power'] = word[4]
            d2['Confidence'] = word[5]
            d2['Surface Temperature (Celcius)'] = word[6]
            d1['Fire Data'] = d1['Fire Data'] + [d2]
    

    # inserting to db
    db.insert_one(d1)
    
    # closing the mongo client
    client.close()
    return 0

# for each rdd in the stream, execute the function sendDb
incoming.foreachRDD(sendDb)
    

# making into printable form and prints for verification
digits = incoming.flatMap(lambda line: line.split("\n"))
digits.pprint()

# starts
ssc.start()

-------------------------------------------
Time: 2018-05-21 13:40:50
-------------------------------------------
fdata,-37.966,145.051,341.8,26.7,78,68
fdata,-35.541,143.311,336.3,62,82,63
fdata,-35.554,143.307,326.8,23.8,67,53
fdata,-35.543,143.316,340.4,84.2,86,67
fdata,-37.708,145.1,327.8,16.2,80,54
cdata,948700,19,56.8,7.9,11.1,   72.0*,  61.9*, 0.00I

-------------------------------------------
Time: 2018-05-21 13:40:51
-------------------------------------------
fdata,-35.646,142.282,305.6,11.8,65,32
fdata,-38.075,143.76,320.1,17.6,72,46
fdata,-37.636,149.33,316.7,37.1,94,43
fdata,-37.624,149.332,306.8,20.8,69,33
fdata,-37.82,142.323,327.6,18.9,62,54
cdata,948700,15,50.7,9.2,13,                                                                                                                                                                                                                                                                                                                  

In [ ]:
# stop everything
# use this for stopping before end
ssc.stop()
sc.stop()

## Test the Data in DB

We can use the below code to test if the data is in the DB

In [16]:
# connect to db
client = MongoClient()
db = client.fit5148_db2
db = db.streamed

# fetch data from db
result = db.find()

# display the data
for ele in result:
    p.pprint(ele)

{'Air Temperature(Celcius)': '19',
 'Fire Data': [{'Confidence': '78',
                'Latitude': '-37.966',
                'Longitude': '145.051',
                'Power': '26.7',
                'Surface Temperature (Celcius)': '68',
                'Surface Temperature (kelvin)': '341.8'},
               {'Confidence': '82',
                'Latitude': '-35.541',
                'Longitude': '143.311',
                'Power': '62',
                'Surface Temperature (Celcius)': '63',
                'Surface Temperature (kelvin)': '336.3'},
               {'Confidence': '67',
                'Latitude': '-35.554',
                'Longitude': '143.307',
                'Power': '23.8',
                'Surface Temperature (Celcius)': '53',
                'Surface Temperature (kelvin)': '326.8'},
               {'Confidence': '86',
                'Latitude': '-35.543',
                'Longitude': '143.316',
                'Power': '84.2',
                'Surface Temperature

## Summary 

Created a stream client which recevies the stram from server and load into the mongo db after converting to suitable format.

The client stops when it receives a finsih signal from server

Also tested the program by checking the data added to DB by querying with pymongo.